In [13]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, classification_report

dataset = pd.read_csv('datasets\story_emotion4.csv', encoding='ISO-8859-1')
stemmer = pd.read_csv('datasets\stem_tl.csv')
word_to_stem = dict(zip(stemmer['word'], stemmer['stem']))

replace_patterns = {
    re.compile(r"\bngayo\'y\b"): 'ngayon ay',
    re.compile(r"\bhangga\'t\b"): 'hanggang',
    re.compile(r"\b\'?y\b"): ' ay',
    re.compile(r"\b\'?t\b"): ' at',
    re.compile(r"\b\'?yan\b"): 'iyan',
    re.compile(r"\b\'?yo\b"): 'iyo',
    re.compile(r"\b\'?yon\b"): 'iyon',
    re.compile(r"\b\'?yun\b"): 'iyun',
    re.compile(r"\b\'?pagkat\b"): 'sapagkat',
    re.compile(r"\b\'?di\b"): 'hindi',
    re.compile(r"\b\'?kaw\b"): "ikaw",
    re.compile(r"\b\'?to\b"): 'ito',
    re.compile(r"\b\'?wag\b"): 'huwag',
    re.compile(r"\bgano\'n\b"): 'ganoon'
}

class_names = {
    1: 'fear',
    2: 'anger',
    3: 'joy',
    4: 'sadness',
    5: 'disgust',
    6: 'surprise'
}

def data_preprocess(text, replace_patterns, word_to_stem):
    text = text.lower()

    for pattern, replacement in replace_patterns.items():
        text = pattern.sub(replacement, text)

    text = re.sub("[^a-zA-Z\s?!.]", '', text)
    tokens = word_tokenize(text)
    text = ' '.join([word_to_stem.get(word, word) for word in tokens])

    return text

dataset['text_preprocessed'] = dataset['text'].apply(data_preprocess, replace_patterns=replace_patterns, word_to_stem=word_to_stem)

class MulticlassSVM:
    def __init__(self, learning_rate=0.01, num_epochs=5, C=1.0):
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.C = C
        self.classes = None
        self.classifiers = {}

    def fit(self, X, Y):
        self.classes = np.unique(Y)

        for i, class_label in enumerate(self.classes):
            binary_labels = np.where(Y == class_label, 1, -1)
            weights, bias = self._train_one_vs_rest(X, binary_labels)
            self.classifiers[class_label] = {"weights": weights, "bias": bias}

    def predict(self, X):
        predictions = np.zeros((X.shape[0], len(self.classes)))

        for i, class_label in enumerate(self.classes):
            weights = self.classifiers[class_label]["weights"]
            bias = self.classifiers[class_label]["bias"]
            predictions[:, i] = np.dot(X, weights) + bias

        predicted_labels = self.classes[np.argmax(predictions, axis=1)]

        return predicted_labels

    def _train_one_vs_rest(self, X, binary_labels):
        num_samples, num_features = X.shape
        weights = np.zeros(num_features)
        bias = 0

        for epoch in range(self.num_epochs):
            for i in range(num_samples):
                if binary_labels[i] * (np.dot(X[i], weights) + bias) < 1:
                    weights = weights + self.learning_rate * (
                        binary_labels[i] * X[i] - 2 * self.C * weights
                    )
                    bias = bias + self.learning_rate * binary_labels[i]

        return weights, bias

vectorizer = CountVectorizer()
tfidf_transformer = TfidfTransformer()

X = dataset['text']
Y = dataset['emotion']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

X_train_tfidf = tfidf_transformer.fit_transform(X_train_vectorized)
X_test_tfidf = tfidf_transformer.transform(X_test_vectorized)

svm_classifier = MulticlassSVM(C=0.1)

svm_classifier.fit(X_train_tfidf.toarray(), Y_train.values)

Y_pred = svm_classifier.predict(X_test_tfidf.toarray())


accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy Manual Implementation of SVM:", accuracy)

print("\nClassification Report:\n", classification_report(Y_test, Y_pred, target_names=class_names.values()))

Accuracy Manual Implementation of SVM: 0.8333333333333334

Classification Report:
               precision    recall  f1-score   support

        fear       0.82      0.90      0.86        40
       anger       0.67      0.90      0.77        40
         joy       0.83      0.75      0.79        40
     sadness       0.97      0.70      0.81        40
     disgust       0.90      0.90      0.90        40
    surprise       0.92      0.85      0.88        40

    accuracy                           0.83       240
   macro avg       0.85      0.83      0.83       240
weighted avg       0.85      0.83      0.83       240



In [16]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

for fold, (train_index, test_index) in enumerate(skf.split(X, Y)):
    X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
    Y_train_fold, Y_test_fold = Y.iloc[train_index], Y.iloc[test_index]

    X_train_vectorized_fold = vectorizer.fit_transform(X_train_fold)
    X_test_vectorized_fold = vectorizer.transform(X_test_fold)

    X_train_tfidf_fold = tfidf_transformer.fit_transform(X_train_vectorized_fold)
    X_test_tfidf_fold = tfidf_transformer.transform(X_test_vectorized_fold)

    svm_classifier = MulticlassSVM(C=0.1)
    svm_classifier.fit(X_train_tfidf_fold.toarray(), Y_train_fold)

    Y_pred_fold = svm_classifier.predict(X_test_tfidf_fold.toarray())

    accuracy_scores.append(accuracy_score(Y_test_fold, Y_pred_fold))
    precision_scores.append(precision_score(Y_test_fold, Y_pred_fold, average='weighted'))
    recall_scores.append(recall_score(Y_test_fold, Y_pred_fold, average='weighted'))
    f1_scores.append(f1_score(Y_test_fold, Y_pred_fold, average='weighted'))

print(f'Average Accuracy: {np.mean(accuracy_scores)}')
print(f'Average Precision: {np.mean(precision_scores)}')
print(f'Average Recall: {np.mean(recall_scores)}')
print(f'Average F1 Score: {np.mean(f1_scores)}')


C:\Users\Anna Margarita\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Anna Margarita\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Anna Margarita\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Anna Margarita\anaconda3\lib\site-packages\sklearn\metrics

Average Accuracy: 0.16666666666666666
Average Precision: 0.02777777777777778
Average Recall: 0.16666666666666666
Average F1 Score: 0.04761904761904761


C:\Users\Anna Margarita\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
